## Packages

In [1]:
import os
import os.path as osp
from pathlib import Path
from time import time
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import librosa

from IPython.display import Audio, clear_output, display

In [2]:
import torch

## Arguments

In [3]:
wavs_dir = "../wavs/"
wavs = [path for path in Path(wavs_dir).rglob("*.wav")]

output_dir = "../outputs/npy2/"
transcript_path = "../outputs/all_transcripts.csv"
transcripts = pd.read_csv(transcript_path)

target_sr = 16000
target_channel = 1  # 1 or 2
pitch_range = ("C2", "C5")

# Re-process the start and end index as the files will be resampled
transcripts["start_idx"] = np.floor(transcripts["start"] * target_sr).astype(int)
transcripts["end_idx"] = np.ceil(transcripts["end"] * target_sr).astype(int)

if not osp.exists(output_dir):
    os.makedirs(output_dir)

## User Defined Functions

In [4]:
def resample(audio, current_sr, target_sr):
    if current_sr == target_sr:
        return audio
    else:
        new_audio = librosa.resample(audio, current_sr, target_sr)
        return new_audio


def rechannel(audio, target_channel):
    current_shape = audio.shape
    if target_channel == 1:
        if len(current_shape) == 1:
            return audio
        else:
            new_audio = librosa.to_mono(audio)
            return new_audio
    else:
        if len(current_shape) == 2:
            return audio
        else:
            new_audio = np.column_stack([audio, audio])
            return new_audio


def extract_pitch(
    audio, pitch_range=("C2", "C5"), impute_val=0.0, log=True
):
    f0, voiced_flags, voiced_probs = librosa.pyin(
        y=audio,
        fmin=librosa.note_to_hz(pitch_range[0]),
        fmax=librosa.note_to_hz(pitch_range[1]),
    )
    if log:
        f0 = np.log2(f0)
    f0_inputed = np.nan_to_num(f0, nan=impute_val)
    return (f0_inputed, voiced_flags, voiced_probs)


def extract_onset(audio, sampling_rate, max_size=5):
    onset_strengths = librosa.onset.onset_strength(y=audio, sr=sampling_rate, max_size=max_size)
    onset_frames = librosa.onset.onset_detect(y=audio, sr=sampling_rate, units="frames")
    onset_flags = np.zeros(onset_strengths.shape[0])
    onset_flags[onset_frames] = 1
    return (onset_strengths, onset_flags)

## Extract Features (One File Sample)

In [5]:
# File Path
file_path = wavs[0]

# Load in audio
audio_array, audio_sr = librosa.load(
    file_path, sr=librosa.core.get_samplerate(file_path)
)

# Pre-process (resample, rechannel)
audio_array = resample(audio_array, audio_sr, target_sr)
audio_array = rechannel(audio_array, target_channel)

In [7]:
file_transcripts = transcripts.loc[transcripts["file"] == file_path.name]

print(file_path.name)
print(f"{len(file_transcripts)} transcript lines")

# Iterate over transcript metadata
# Segment clip by start and end indices

shapes = []
st = time()
for x in file_transcripts.iterrows():
    row = x[1]
    clear_output(wait=True)
    print(
        "Item:",
        x[0],
        " |  Line:",
        row["line"],
        " |  Progress:",
        f"{round(100*(x[0]/file_transcripts.shape[0]))}%",
    )
    print(f"Elapsed Time: {round(time() - st, 2)}s")

    segment = audio_array[row["start_idx"] : row["end_idx"]]
    f0_inputed, voiced_flags, voiced_probs = extract_pitch(
        segment, pitch_range, log=True
    )
    onset_strengths, onset_flags = extract_onset(segment, target_sr)
    full_array = np.column_stack(
        (f0_inputed, voiced_flags, voiced_probs, onset_strengths, onset_flags)
    )
    np.save(osp.join(output_dir, f"{file_path.name}_{row['line']}.npy"), full_array)

    shapes.append(full_array.shape)

# For record keeping, used to decide max_lens
np.save(
    osp.join(output_dir, f"{file_path.name}_shapes.npy"),
    np.array([s[0] for s in shapes]),
)
print(f"Total Time: {round(time() - st, 2)}s")

Item: 2  |  Line: 7  |  Progress: 5%
Elapsed Time: 3.7s


KeyboardInterrupt: 

## Loop Over All

In [6]:
for i,f in enumerate (wavs[1:3]):
    # Load in audio
    audio_array, audio_sr = librosa.load(
        f, sr=librosa.core.get_samplerate(f)
    )

    # Pre-process (resample, rechannel)
    audio_array = resample(audio_array, audio_sr, target_sr)
    audio_array = rechannel(audio_array, target_channel)

    file_transcripts = transcripts.loc[transcripts["file"] == f.name]



    # Iterate over transcript metadata
    # Segment clip by start and end indices

    shapes = []
    st = time()
    for j,x in enumerate(file_transcripts.iterrows()):
        row = x[1]
        clear_output(wait=True)
        print(f.name)
        print(f"{len(file_transcripts)} transcript lines")
        print(
            "Item:",
            x[0],
            " |  Line:",
            row["line"],
            " |  Progress:",
            f"{round(100*(j/file_transcripts.shape[0]))}%",
        )
        print(f"Elapsed Time: {round(time() - st, 2)}s")

        segment = audio_array[row["start_idx"] : row["end_idx"]]
        try:
            f0_inputed, voiced_flags, voiced_probs = extract_pitch(
                segment, pitch_range, log=True
            )
            onset_strengths, onset_flags = extract_onset(segment, target_sr)
            full_array = np.column_stack(
                (f0_inputed, voiced_flags, voiced_probs, onset_strengths, onset_flags)
            )
            np.save(osp.join(output_dir, f"{row['file']}_{row['line']}.npy"), full_array)
        except:
            pass
            # Need to do something with the transcript that have bad segments
        shapes.append(full_array.shape)

    # For record keeping, used to decide max_lens
    np.save(
        osp.join(output_dir, f"{f.name}_shapes.npy"),
        np.array([s[0] for s in shapes]),
    )
    print(f"Total Time: {round(time() - st, 2)}s")


18-1447.wav
41 transcript lines
Item: 127  |  Line: 256  |  Progress: 310%
Elapsed Time: 85.22s
Total Time: 86.72s


In [8]:
x[0]

127

In [11]:
file_transcripts

,Unnamed: 0,file,line,speaker,start,end,duration,sample_rate,channels,start_idx,end_idx,word_count,text
87,0,18-1447.wav,2,John_G_Roberts_Jr,136.72,161.64,24.92,16000,1,2187520,2586240,57,"Mr. Silbert, I want to pick up on that very la..."
88,1,18-1447.wav,7,Clarence_Thomas,223.68,244.12,20.44,16000,1,3578880,3905920,44,"Yes, thank you, Mr. Chief Justice. Counsel, I ..."
89,2,18-1447.wav,19,Stephen_G_Breyer,381.96,422.28,40.32,16000,1,6111360,6756480,102,I'd like to -- to pick up on that last questio...
90,3,18-1447.wav,24,Samuel_A_Alito_Jr,529.00,561.36,32.36,16000,1,8464000,8981760,61,Section 1606 on which you rely makes a foreign...
91,4,18-1447.wav,26,Samuel_A_Alito_Jr,627.28,646.76,19.48,16000,1,10036480,10348160,40,"One other question. If we were to rule, hypoth..."
92,5,18-1447.wav,32,Sonia_Sotomayor,678.92,708.00,29.08,16000,1,10862720,11328000,54,"Counsel -- counsel, I -- I don't understand ho..."
93,6,18-1447.wav,34,Sonia_Sotomayor,723.36,742.92,19.56,16000,1,11573760,11886720,46,"I thought that -- counsel, I understand that p..."
94,7,18-1447.wav,38,Sonia_Sotomayor,751.88,780.72,28.84,16000,1,12030080,12491520,59,-- one final point. As I read the record below...
95,8,18-1447.wav,42,Elena_Kagan,820.64,854.16,33.52,16000,1,13130240,13666560,85,"Mr. Silbert, you've referred a couple of times..."
96,9,18-1447.wav,48,Elena_Kagan,935.52,969.20,33.68,16000,1,14968320,15507200,100,You said you're not -- we're not going back to...
